In [149]:
import pandas as pd

In [150]:
hitter = pd.read_csv('../선수데이터(전처리완료)/모델링용ver3/타자(모델링용)_train_ver3.csv')
fa_ids = hitter.loc[hitter['FA여부'] == 1, 'ID'].unique()
fa_hitter = hitter[hitter['ID'].isin(list(fa_ids))].reset_index(drop=True)
nonfa_hitter = hitter[~hitter['ID'].isin(list(fa_ids))].reset_index(drop=True)

pitcher = pd.read_csv('../선수데이터(전처리완료)/모델링용ver3/투수(모델링용)_train_ver3.csv')
fa_ids = pitcher.loc[pitcher['FA여부'] == 1, 'ID'].unique()
fa_pitcher = pitcher[pitcher['ID'].isin(list(fa_ids))].reset_index(drop=True)
nonfa_pitcher = pitcher[~pitcher['ID'].isin(list(fa_ids))].reset_index(drop=True)

In [155]:
fa_pitcher.loc[fa_pitcher['선수명'] == '윤석민'][['연도','FA여부','연봉']]

,연도,FA여부,연봉
538,2005,0,5914.939777
539,2006,0,11537.423238
540,2007,0,13471.578434
541,2008,0,21452.991453
542,2009,0,25471.878398
543,2010,0,21146.995787
544,2011,0,41388.587546
545,2012,0,40857.131201
546,2015,1,200114.243241


#### 2. 1차 FA기간, 2차 FA기간을 원핫인코딩으로 표기 
 - [선수명, 팀명, 연도] 기준 MERGE(확인결과 중복존재하지 않음)

In [147]:
def change_teamname(x):
    
    """
    기아 -> KIA
    히어로즈, 넥센, 키움 -> 히어로즈/넥센/키움
    kt -> KT
    우리, 현대 -> 우리/현대
    """
    
    if x == '기아':
        x = 'KIA'
    elif x == 'kt':
        x = 'KT'
    elif (x == '히어로즈') or (x == '넥센') or (x == '키움') or (x == '우리'):
        x = '우리/히어로즈/넥센/키움'
        
    return x

fa_data = pd.read_csv('../FA정보데이터/FA_정보.csv').dropna().reset_index(drop=True)#[['선수명','연도','계약팀','계약기간']]
fa_data = fa_data.rename(columns = {'계약팀':'팀명'})
#fa_data['연도'] = fa_data['연도'] - 1
fa_data['팀명'] = fa_data['팀명'].apply(lambda x: change_teamname(x))
fa_data['계약기간'] = fa_data['계약기간'].apply(lambda x: int(x.replace('년','')))
fa_data

,선수명,생년월일,연도,이전팀,팀명,계약기간,계약총액,계약평균액
0,오주원,1985-03-31,2020,키움,우리/히어로즈/넥센/키움,2,"70,000","35,000"
1,김태균,1982-05-29,2020,한화,한화,1,"100,000","100,000"
2,오재원,1985-02-09,2020,두산,두산,3,"190,000","63,333"
3,김태군,1989-12-30,2020,NC,NC,4,"130,000","32,500"
4,이성열,1984-07-13,2020,한화,한화,2,"140,000","70,000"
...,...,...,...,...,...,...,...,...
225,안경현,1970-02-13,2003,두산,두산,4,"150,000","37,500"
226,양준혁,1969-05-26,2002,LG,삼성,4,"272,000","68,000"
227,김민재,1973-01-03,2002,롯데,SK,4,"100,000","25,000"
228,전준호,1969-02-15,2002,현대,현대,3,"120,000","40,000"


In [154]:
fa_data[fa_data['선수명'] == '윤석민']

,선수명,생년월일,연도,이전팀,팀명,계약기간,계약총액,계약평균액
86,윤석민,1986-07-24,2015,볼티모어,KIA,4,"900,000","225,000"
106,윤석민,1986-07-24,2014,KIA,볼티모어,3,"575,000","191,667"


In [138]:
fa_hitter = pd.merge(fa_hitter, fa_data, on=['선수명', '팀명', '연도'], how='left')
fa_pitcher = pd.merge(fa_pitcher, fa_data, on=['선수명', '팀명', '연도'], how='left')

In [144]:
tmp = fa_hitter.loc[fa_hitter['FA여부'] == 1]
tmp[tmp['계약기간'].isnull()]

,ID,선수명,데뷔년도,팀명,연도,타율,경기,타석,타수,득점,...,뉴스개수(누적정규화),수상횟수,누적타석,포지션 엔트로피,출생연도,출생월,출생일,나이,연봉,계약기간
160,94415,김태균,1994,SK,2005,0.237,110,272,232,24,...,0.309659,0,783,0.180692,1971,8,19,35,13144.310616,NaN
192,96610,장성호,1996,한화,2010,0.245,74,279,237,34,...,1.641677,0,4198,0.000000,1977,10,18,34,22259.995566,NaN
263,96723,홍원기,1996,현대,2006,0.270,56,173,148,16,...,0.261104,0,1169,0.794100,1973,6,5,34,8973.551408,NaN
353,94836,조원우,1994,한화,2005,0.302,95,392,351,43,...,0.259352,0,1702,0.000000,1971,4,8,35,29574.698885,NaN
643,72860,이대수,2001,SK,2014,0.160,15,28,25,1,...,0.919620,0,3021,1.239746,1981,8,21,34,52710.000000,NaN
715,91470,신동주,1991,KIA,2005,0.094,23,35,32,2,...,0.093706,0,664,0.000000,1972,12,14,34,21468.602529,NaN
720,97644,김창희,1997,삼성,2006,0.221,114,351,307,35,...,0.215582,0,1643,0.000000,1973,7,3,34,16665.166900,NaN
785,73136,이성열,2003,한화,2015,0.250,101,268,232,35,...,0.825796,0,2781,0.094084,1984,7,13,32,26101.812420,NaN
937,77463,채태인,2007,롯데,2018,0.293,130,420,376,44,...,1.997106,0,4029,0.000000,1982,10,11,37,40217.453505,NaN
964,77564,김민성,2007,LG,2019,0.260,107,401,361,44,...,1.699006,0,4741,0.000000,1988,12,17,32,50000.000000,NaN


#### 1. FA연도 1년씩 shift

In [127]:
### 타자
fa_hitter_final = pd.DataFrame()

for id in fa_hitter['ID'].unique():
    tmp = hitter[hitter['ID'] == id].reset_index(drop=True)
    years = list(tmp.loc[tmp['FA여부'] == 1, '연도'])
    previous_years = [year-1 for year in years]
    
    tmp['FA여부'] = 0
    tmp.loc[tmp['연도'].isin(previous_years), 'FA여부'] = 1
    
    fa_hitter_final = pd.concat([fa_hitter_final, tmp]).reset_index(drop=True)
    
### 투수
fa_pitcher_final = pd.DataFrame()

for id in fa_pitcher['ID'].unique():
    tmp = pitcher[pitcher['ID'] == id].reset_index(drop=True)
    years = list(tmp.loc[tmp['FA여부'] == 1, '연도'])
    previous_years = [year-1 for year in years]
    
    tmp['FA여부'] = 0
    tmp.loc[tmp['연도'].isin(previous_years), 'FA여부'] = 1
    
    fa_pitcher_final = pd.concat([fa_pitcher_final, tmp]).reset_index(drop=True)

138

In [79]:
for id in fa_pitcher_final['ID'].unique():
    tmp = fa_pitcher_final[fa_pitcher_final['ID'] == id].reset_index(drop=True)
    if tmp['FA여부'].sum() == 0:
        print(id)

75620
98259


In [84]:
fa_pitcher_final[fa_pitcher_final['ID'] == 75620].reset_index(drop=True)[['연도','FA여부', '연봉']]

,연도,FA여부,연봉
0,2005,0,5914.939777
1,2006,0,11537.423238
2,2007,0,13471.578434
3,2008,0,21452.991453
4,2009,0,25471.878398
5,2010,0,21146.995787
6,2011,0,41388.587546
7,2012,0,40857.131201
8,2015,0,200114.243241


In [86]:
fa_data[fa_data['선수명'] == '윤석민']

,선수명,연도,팀명,계약기간
86,윤석민,2014,볼티모어,4
106,윤석민,2013,KIA,3
